# Lab 2: Localization of Wireless Cameras

Important Notes:

-The xloc and yloc are all flipped in the data given to us

-Units are in inches.

-We need to localize 3 devices with the following MAC addresses:
    1. f8:cf:c5:97:e0:9e
    2. ec:d0:9f:db:e8:1f
    3. 80:e6:50:1b:a7:80
    4. Ground truth of device with MAC 44:91:60:d3:d6:94 is (-22, 162)
  Plan:
    1. We sort the all datasets into 4 sub-datasets for each MAC.
    2. The algorithm will use the log distance path loss model equation shown in detail in Lecture 7 Slides.
    3. Test the algorithm for the ground truth device.
    4. Extract location of other 3 devices.

In [17]:
import numpy as np, pandas as pd, json, os

## 1. Take all datasets and sort them into 4 tables, one for each MAC address

In [18]:
cf = []
d0 = []
e6 = [] #initializaing empty arrays for each mac address
gt = [] #ground-truth device
for file in os.listdir('final_lab2_data'): #takes data from every file in folder
    filename = os.fsdecode(file)
    path = 'final_lab2_data/' + filename
    with open(path, 'r') as f:
        data = (json.loads(f.read().replace('\'','\"')))
    for x in range(0, len(data)):
        entry = data[x]
        if entry['mac'] == 'f8:cf:c5:97:e0:9e':
            cf.append({'loc_x': entry['loc_y'], 'loc_y': entry['loc_x'], 'rss:': entry['rss']})
        elif entry['mac'] == 'ec:d0:9f:db:e8:1f':
            d0.append({'loc_x': entry['loc_y'], 'loc_y': entry['loc_x'], 'rss:': entry['rss']})
        elif entry['mac'] == '80:e6:50:1b:a7:80':
            e6.append({'loc_x': entry['loc_y'], 'loc_y': entry['loc_x'], 'rss:': entry['rss']})
        elif entry['mac'] == '44:91:60:d3:d6:94':
            gt.append({'loc_x': entry['loc_y'], 'loc_y': entry['loc_x'], 'rss:': entry['rss']})
cfdf = pd.DataFrame(cf)
d0df = pd.DataFrame(d0)
e6df = pd.DataFrame(e6)
gtdf = pd.DataFrame(gt)

## 2. Find linear fit line for each device using equation in Lecture 7. Deduce constants from fit.